This file was used to blend our financial data sources

In [54]:
import numpy as np #linear algebra
import pandas as pd #data processing 
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

# Importing into dataframe
df_Bom_MovieGross = pd.read_csv("./Prj_Data/DownLoadedData_FlatIron/bom.movie_gross.csv.gz", delimiter=',', encoding='ANSI')
df_TN_Movie_Budgets = pd.read_csv("./Prj_Data/DownLoadedData_FlatIron/tn.movie_budgets.csv.gz", delimiter=',')
df_TN_Movie_Budgets = pd.read_csv("./Prj_Data/DownLoadedData_FlatIron/tn.movie_budgets.csv.gz", delimiter=',')
df_Scrp_Financials = pd.read_excel("./Prj_Data/ImdbScrapingData/df_Financials_SummaryAndDetail_Imdb_Dom.xlsx")
df_IMDB_Akas_english = pd.read_excel("./Prj_Data/DownLoadedData_Imdb/df_IMDB_Akas_english.xlsx")
df_IMDB_InflationAdjuster = pd.read_excel("./Prj_Data/ImdbScrapingData/InflationAdjuster.xlsx")


In [55]:
#PREPARE THE FINACIAL DATA:  WE USED 2 SOURCES,1 FROM FLATIRON DOWNLOAD, 2 FROM SCRAPING IMDB Site :https://www.boxofficemojo.com/
#****************************************************************************************
#prepraring to work with Movie Gross Df, cleaning up data and create right datatypes
df_Bom_MovieGross.name = "df_Bom_MovieGross"
df_Bom_MovieGross['foreign_gross'] = pd.to_numeric(df_Bom_MovieGross['foreign_gross'],errors='coerce')
df_Bom_MovieGross['year_str_BOM'] = df_Bom_MovieGross['year'].astype(str)
df_Bom_MovieGross['year'] = df_Bom_MovieGross['year'].astype(int)
df_Bom_MovieGross_values = {'domestic_gross':0, 'foreign_gross': 0}
df_Bom_MovieGross.fillna(value=df_Bom_MovieGross_values, inplace=True)
df_Bom_MovieGross["wwg_calc_BOM"] = 0 
df_Bom_MovieGross['title'] = df_Bom_MovieGross['title'].str.title() #******KEY STEP
df_Bom_MovieGross["titleyear"] = df_Bom_MovieGross['title'] + df_Bom_MovieGross['year_str_BOM']
df_Bom_MovieGross["wwg_calc_BOM"] = (df_Bom_MovieGross['domestic_gross'] + df_Bom_MovieGross['foreign_gross'])

#Renaming to aid in consolidating between the three sources
df_Bom_MovieGross.rename(columns={"foreign_gross": "fg_BOM", "domestic_gross": "dg_BOM","year": "year_BOM",
                                             "title":"title_BOM","studio":"studio_BOM" },inplace=True)
#-------------------------------------------------------------

#prepraring to work with Movie budgets, cleaning up data and create right datatypes
df_TN_Movie_Budgets.name = "df_TN_Movie_Budgets"
df_TN_Movie_Budgets[df_TN_Movie_Budgets.columns[1:]] = df_TN_Movie_Budgets[df_TN_Movie_Budgets.columns[1:]].replace('[\$,]', '', regex=True)
df_TN_Movie_Budgets['production_budget'] = pd.to_numeric(df_TN_Movie_Budgets['production_budget'],errors='coerce')
df_TN_Movie_Budgets['domestic_gross'] = pd.to_numeric(df_TN_Movie_Budgets['domestic_gross'],errors='coerce')
df_TN_Movie_Budgets['worldwide_gross'] = pd.to_numeric(df_TN_Movie_Budgets['worldwide_gross'],errors='coerce')

df_TN_Movie_Budgets_values = {'domestic_gross': 0, 'year':0 , 'worldwide_gross': 0}
df_TN_Movie_Budgets.fillna(value=df_TN_Movie_Budgets_values, inplace=True)
df_TN_Movie_Budgets["fg_calc"] = 0

df_TN_Movie_Budgets['movie'] = df_TN_Movie_Budgets['movie'].str.title() #******KEY STEP
df_TN_Movie_Budgets["year"] = df_TN_Movie_Budgets['release_date'].str[-4:].astype(int)
df_TN_Movie_Budgets["year_str_TN"] = df_TN_Movie_Budgets['release_date'].str[-4:]
df_TN_Movie_Budgets["titleyear"] = df_TN_Movie_Budgets['movie'] + df_TN_Movie_Budgets['year_str_TN']
df_TN_Movie_Budgets["fg_calc_TN"] = (df_TN_Movie_Budgets['worldwide_gross'] - df_TN_Movie_Budgets['domestic_gross'])
df_TN_Movie_Budgets = df_TN_Movie_Budgets.drop('id', axis = 1)
df_TN_Movie_Budgets.rename(columns={"domestic_gross": "dg_TN", "worldwide_gross": "wwg_TN","year": "year_TN", 
                                    "production_budget": "pb_TN" ,"movie":"title_TN","release_date":"rd_TN" }, inplace=True)

#needed to do this steep to collapse data given there are dups in a few movienames and titles - ?bluebeard
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())
q3 = """SELECT titleyear as titleyear, max(title_TN) as title_TN, max(year_TN) as year_TN, min(year_str_TN) as year_str_TN, 
min(rd_TN) as rd_TN, Sum(pb_TN) as pb_TN,sum(dg_TN) as dg_TN, sum(wwg_TN) as wwg_TN , sum(fg_calc_TN) as fg_calc_TN
        FROM df_TN_Movie_Budgets
        GROUP BY titleyear, title_TN
        """
df_TN_Movie_Budgets = pysqldf(q3)
#-------------------------------------------------------------

# Finanally merger data from flatiron
df_fI_Financials = df_TN_Movie_Budgets.merge(df_Bom_MovieGross, 
     on='titleyear', how='outer', indicator='Combing_FL_Financials', suffixes=('_TN', '_Bom'))

# Add tconst Key to FLatIronTables then drop 
df_FI_financials_With_tconst = df_fI_Financials.merge(df_IMDB_Akas_english[['tconst','titleyear']], on='titleyear', how="inner", indicator='Adding tconst', suffixes=('_fi', '_im'))


In [56]:
# df_FI_financials_With_tconst.info()

In [57]:
# df_FI_financials_With_tconst.nunique()

In [58]:
#USEDTO CHECK FOR MERGING ERRORS
# # df_FI_financials_With_tconst[["title_TN","title_BOM","title_FI",]].sort_values(by="title_FI", ascending=False)
# df_FI_financials_With_tconst[["year_TN","year_BOM","year_str_FI",]].sort_values(by="year_str_FI", ascending=False)
# df_FI_financials_With_tconst[["year_TN","year_BOM","year_FI",]].sort_values(by="year_FI", ascending=False)
# # df_Bom_MovieGross[["studio","studio_short","Studio_Desc"]].sort_values(by="title_FI", ascending=False)

In [59]:
#Use all data scraped from IMDB as the default for domestic, foreign ww sales as well as budget
df_FI_financials_With_tconst.title_TN.fillna("0", inplace=True)
df_FI_financials_With_tconst.year_str_TN.fillna("0", inplace=True)
df_FI_financials_With_tconst.year_TN.fillna(0, inplace=True)

df_FI_financials_With_tconst.title_BOM.fillna("0", inplace=True)
df_FI_financials_With_tconst.year_str_BOM.fillna("0", inplace=True)
df_FI_financials_With_tconst.year_BOM.fillna(0, inplace=True)

df_FI_financials_With_tconst['title_FI'] = df_FI_financials_With_tconst['title_TN']
df_FI_financials_With_tconst['year_str_FI'] = df_FI_financials_With_tconst['year_str_TN']
df_FI_financials_With_tconst['year_FI'] = df_FI_financials_With_tconst['year_TN']

df_FI_financials_With_tconst['title_FI'] = df_FI_financials_With_tconst.title_FI.fillna("0").astype(str)
df_FI_financials_With_tconst['year_str_FI'] = df_FI_financials_With_tconst.year_str_FI.fillna("0").astype(str)
df_FI_financials_With_tconst['year_FI'] = df_FI_financials_With_tconst.year_FI.fillna(0).astype(int)

df_FI_financials_With_tconst['title_FI'] = df_FI_financials_With_tconst.apply(lambda x: x.title_BOM if x.title_FI == "0" else x.title_FI, axis=1)
df_FI_financials_With_tconst['year_str_FI'] = df_FI_financials_With_tconst.apply(lambda x: x.year_str_BOM if x.year_str_FI == "0" else x.year_str_FI, axis=1)
df_FI_financials_With_tconst['year_FI'] = df_FI_financials_With_tconst.apply(lambda x: x.year_BOM if x.year_FI == 0 else x.year_FI, axis=1)


In [60]:
df_FI_financials_With_tconst.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2609 entries, 0 to 2608
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   titleyear              2609 non-null   object  
 1   title_TN               2609 non-null   object  
 2   year_TN                2609 non-null   float64 
 3   year_str_TN            2609 non-null   object  
 4   rd_TN                  1983 non-null   object  
 5   pb_TN                  1983 non-null   float64 
 6   dg_TN                  1983 non-null   float64 
 7   wwg_TN                 1983 non-null   float64 
 8   fg_calc_TN             1983 non-null   float64 
 9   title_BOM              2609 non-null   object  
 10  studio_BOM             1646 non-null   object  
 11  dg_BOM                 1647 non-null   float64 
 12  fg_BOM                 1647 non-null   float64 
 13  year_BOM               2609 non-null   float64 
 14  year_str_BOM           2609 non-null   o

In [61]:
# df_FI_financials_With_tconst.sort_values(by="year_FI").head(100)

In [62]:
#DROP UNNEEDED FIELDS
df_FI_financials_With_tconst.drop(["year_TN","year_BOM","year_TN","year_BOM", "year_str_TN","year_str_BOM","Combing_FL_Financials"], axis=1, inplace=True)

In [63]:
# df_Scrp_Financials.info()

In [64]:
#Merg flatiron financials with scraping finacials IMDB Site :https://www.boxofficemojo.com/
#****************************************************************************************
#prepraring to work with Movie Gross Df, cleaning up data and create right datatypes
df_MasterFinancials = df_Scrp_Financials.merge(df_FI_financials_With_tconst, 
                                               on='tconst', how='outer', suffixes=('_IM', '_fl'))
fieldsToConvert = {'dg_IMDB': 0, 'fg_IMDB':0 , 'ww_IMDB': 0, 'Domestic Opening': 0, 'pb_IMDB':0 , 'pb_TN': 0,
                       'dg_TN': 0, 'wwg_TN':0 , 'fg_calc_TN': 0,'dg_BOM': 0, 'fg_BOM':0 , 'wwg_calc_BOM': 0,
                      'wg_calc_BOM': 0}
df_MasterFinancials.fillna(value=fieldsToConvert, inplace=True)

In [65]:
# df_MasterFinancials.info()

In [66]:
#Clean up dataframe - consolidate, remove unwanted columns, all remove metadata from financials
df_MasterFinancials = df_MasterFinancials.reindex(sorted(df_MasterFinancials.columns), axis=1)
col_names = ['tconst','Adding tconst', 'Combing_FL_Financials',
       'Genres_IMDB', 'MPAA', 'Running Time_IMDB', 'Unnamed: 0', 'Domestic Opening','dg_BOM',
       'dg_IMDB', 'dg_TN', 'fg_BOM', 'fg_IMDB', 'fg_calc_TN', 'genres',
       'isAdult', 'mergingfinancials', 'originalTitle', 'pb_IMDB', 'pb_TN',
       'rd_IMDB', 'rd_TN', 'runtimeMinutes', 'studio_BOM', 'studio_IMDB',
        'title_BOM', 'title_IMDB', 'title_TN', 'titleyear_IM',
       'titleyear_fl', 'wwg_calc_BOM', 'ww_IMDB', 'wwg_TN',
       'year', 'year_BOM', 'year_TN']
df_MasterFinancials.reindex(columns=col_names)


# df_MasterFinancials.drop(["Adding tconst", 'Unnamed: 0', 'isAdult', 'mergingfinancials'], axis=1,inplace = True )

df_MasterFinancials_Only = df_MasterFinancials[['tconst','Domestic Opening','dg_BOM', 'dg_TN', 'dg_IMDB',
                                                'fg_BOM', 'fg_calc_TN','fg_IMDB', 'pb_IMDB', 'pb_TN',
                                                'title_BOM', 'title_IMDB', 'title_TN', 'titleyear_IM','titleyear_fl',
                                                'wwg_calc_BOM', 'wwg_TN','ww_IMDB','year',"year_FI"]]


In [67]:
#Use all data scraped from IMDB as the default for domestic, foreign ww sales as well as budget
df_MasterFinancials_Only['WW_Gross'] = df_MasterFinancials_Only['ww_IMDB'].apply(lambda x: x if x>0 else 0)
df_MasterFinancials_Only['Dom_Gross'] = df_MasterFinancials_Only['dg_IMDB'].apply(lambda x: x if x>0 else 0)
df_MasterFinancials_Only['Frgn_Gross'] = df_MasterFinancials_Only['fg_IMDB'].apply(lambda x: x if x>0 else 0)
df_MasterFinancials_Only['P_Cost'] = df_MasterFinancials_Only['pb_IMDB'].apply(lambda x: x if x>0 else 0)
df_MasterFinancials_Only['year'] = df_MasterFinancials_Only['year'].apply(lambda x: x if x>0 else 0)



#fill in the blanks from max valus from the flatIron finance tables
df_MasterFinancials_Only['dg_Fl_Max'] = df_MasterFinancials_Only[["dg_BOM", "dg_TN"]].max(axis=1)
df_MasterFinancials_Only['fg_Fl_Max'] = df_MasterFinancials_Only[["fg_BOM", "fg_calc_TN"]].max(axis=1)
df_MasterFinancials_Only['ww_Fl_Max'] = df_MasterFinancials_Only['dg_Fl_Max']+df_MasterFinancials_Only['fg_Fl_Max']

df_MasterFinancials_Only['WW_Gross'] = df_MasterFinancials_Only.apply(lambda x: x.ww_Fl_Max if x.WW_Gross==0 else x.WW_Gross, axis=1)
df_MasterFinancials_Only['Dom_Gross'] = df_MasterFinancials_Only.apply(lambda x: x.dg_Fl_Max if x.Dom_Gross==0 else x.Dom_Gross, axis=1)
df_MasterFinancials_Only['Frgn_Gross'] = df_MasterFinancials_Only.apply(lambda x: x.fg_Fl_Max if x.Frgn_Gross==0 else x.Frgn_Gross, axis=1)
df_MasterFinancials_Only['P_Cost'] = df_MasterFinancials_Only.apply(lambda x: x.pb_TN if x.P_Cost==0 else x.P_Cost, axis=1)
df_MasterFinancials_Only['P_Cost']=df_MasterFinancials_Only['P_Cost'].replace(0,df_MasterFinancials_Only.P_Cost.mean())

df_MasterFinancials_Only['year'] = df_MasterFinancials_Only['year'].fillna(0)
df_MasterFinancials_Only['year'] = df_MasterFinancials_Only.apply(lambda x: x.year_FI if x.year==0 else x.year, axis=1)



df_MasterFinancials_Only['titleyear'] = df_MasterFinancials_Only["titleyear_IM"]
df_MasterFinancials_Only.titleyear.fillna(df_MasterFinancials_Only["titleyear_fl"],inplace=True)

# df_MasterFinancials_MetaData['titleyear'] = df_MasterFinancials_Only["titleyear_IM"]
# df_MasterFinancials_MetaData.titleyear.fillna(df_MasterFinancials_Only["titleyear_fl"],inplace=True)



<ipython-input-67-e68ce41ead83>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_MasterFinancials_Only['WW_Gross'] = df_MasterFinancials_Only['ww_IMDB'].apply(lambda x: x if x>0 else 0)
<ipython-input-67-e68ce41ead83>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_MasterFinancials_Only['Dom_Gross'] = df_MasterFinancials_Only['dg_IMDB'].apply(lambda x: x if x>0 else 0)
<ipython-input-67-e68ce41ead83>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

In [68]:
df_MasterFinancials_Only.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5540 entries, 0 to 5539
Data columns (total 28 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   tconst            5540 non-null   object 
 1   Domestic Opening  5540 non-null   float64
 2   dg_BOM            5540 non-null   float64
 3   dg_TN             5540 non-null   float64
 4   dg_IMDB           5540 non-null   float64
 5   fg_BOM            5540 non-null   float64
 6   fg_calc_TN        5540 non-null   float64
 7   fg_IMDB           5540 non-null   float64
 8   pb_IMDB           5540 non-null   float64
 9   pb_TN             5540 non-null   float64
 10  title_BOM         2609 non-null   object 
 11  title_IMDB        4920 non-null   object 
 12  title_TN          2609 non-null   object 
 13  titleyear_IM      4920 non-null   object 
 14  titleyear_fl      2609 non-null   object 
 15  wwg_calc_BOM      5540 non-null   float64
 16  wwg_TN            5540 non-null   float64


In [69]:
# df_MasterFinancials_Only.sort_values(by="year").head(100)

In [70]:
df_MasterFinancials_Only = df_MasterFinancials_Only[['tconst','P_Cost','Dom_Gross','Frgn_Gross', 'WW_Gross', 'Domestic Opening','year']]

In [71]:
#____________________Add Justments for Inflation and Calculate ROI_______________________________________________

In [72]:
df_MasterFinancials_Only = df_MasterFinancials_Only.merge(df_IMDB_InflationAdjuster, on="year", how="left")

In [73]:
df_MasterFinancials_Only.head()

,tconst,P_Cost,Dom_Gross,Frgn_Gross,WW_Gross,Domestic Opening,year,TicketPrice,EstInflation,Multiplier
0,tt0120667,100000000.0,154696080.0,178839854.0,333535934.0,56061504.0,2005.0,6.41,0.461778,1.461778
1,tt0121164,30000000.0,53359111.0,64731725.0,118090836.0,388166.0,2005.0,6.41,0.461778,1.461778
2,tt0121766,113000000.0,380270577.0,488119983.0,868390560.0,108435841.0,2005.0,6.41,0.461778,1.461778
3,tt0200465,20000000.0,30060660.0,34767761.0,64828421.0,5935256.0,2008.0,7.18,0.305014,1.305014
4,tt0206634,76000000.0,35552383.0,35043081.0,70595464.0,501003.0,2006.0,6.55,0.430534,1.430534


In [74]:
df_MasterFinancials_Only.columns

Index(['tconst', 'P_Cost', 'Dom_Gross', 'Frgn_Gross', 'WW_Gross',
       'Domestic Opening', 'year', 'TicketPrice', 'EstInflation',
       'Multiplier'],
      dtype='object')

In [75]:
df_MasterFinancials_Only["adj_P_Cost"] = df_MasterFinancials_Only["P_Cost"] * df_MasterFinancials_Only["Multiplier"]
df_MasterFinancials_Only["adj_Frgn_Gross"] = df_MasterFinancials_Only["Frgn_Gross"] * df_MasterFinancials_Only["Multiplier"]
df_MasterFinancials_Only["adj_WW_Gross"] = df_MasterFinancials_Only["WW_Gross"] * df_MasterFinancials_Only["Multiplier"]
df_MasterFinancials_Only["Profits"] = df_MasterFinancials_Only["WW_Gross"] - df_MasterFinancials_Only["P_Cost"]
df_MasterFinancials_Only["adj_Profits"] = df_MasterFinancials_Only["Profits"] * df_MasterFinancials_Only["Multiplier"]

df_MasterFinancials_Only["ROI"] = df_MasterFinancials_Only.apply(lambda row : ((row.WW_Gross-row.P_Cost)/row.P_Cost) if row.WW_Gross > 0 else 0, axis=1)


In [76]:
df_MasterFinancials_Only.head()

,tconst,P_Cost,Dom_Gross,Frgn_Gross,WW_Gross,Domestic Opening,year,TicketPrice,EstInflation,Multiplier,adj_P_Cost,adj_Frgn_Gross,adj_WW_Gross,Profits,adj_Profits,ROI
0,tt0120667,100000000.0,154696080.0,178839854.0,333535934.0,56061504.0,2005.0,6.41,0.461778,1.461778,1.461778e+08,2.614242e+08,4.875556e+08,233535934.0,3.413778e+08,2.335359
1,tt0121164,30000000.0,53359111.0,64731725.0,118090836.0,388166.0,2005.0,6.41,0.461778,1.461778,4.385335e+07,9.462344e+07,1.726226e+08,88090836.0,1.287693e+08,2.936361
2,tt0121766,113000000.0,380270577.0,488119983.0,868390560.0,108435841.0,2005.0,6.41,0.461778,1.461778,1.651810e+08,7.135233e+08,1.269395e+09,755390560.0,1.104214e+09,6.684872
3,tt0200465,20000000.0,30060660.0,34767761.0,64828421.0,5935256.0,2008.0,7.18,0.305014,1.305014,2.610028e+07,4.537241e+07,8.460199e+07,44828421.0,5.850171e+07,2.241421
4,tt0206634,76000000.0,35552383.0,35043081.0,70595464.0,501003.0,2006.0,6.55,0.430534,1.430534,1.087206e+08,5.013033e+07,1.009892e+08,-5404536.0,-7.731374e+06,-0.071112


In [77]:
df_MasterFinancials_Only.to_excel("df_Movie_Financials.xlsx")

In [ ]:
#THIS CODE IS FOR CHECKING THE SYNCHING STEP
# df_MasterFinancials_MetaData[["title_TN","title_BOM","title_IMDB","title_fin"]].sort_values(by="title_fin", ascending=False)
# df_MasterFinancials_MetaData[["genres_fin","genres",]].sort_values(by="genres_fin", ascending=False)
# df_MasterFinancials_MetaData[["year_BOM","year_TN","year"]].sort_values(by="year", ascending=False)
# df_MasterFinancials_MetaData[["titleyear_fl","titleyear_IM", "titleyear_fin"]].sort_values(by="titleyear_fin", ascending=False)
# df_MasterFinancials_MetaData[["Running Time_IMDB","runtimeMinutes", "RunningTime_fin"]].sort_values(by="Running Time_IMDB", ascending=False)
#df_MasterFinancials_MetaData[["rd_IMDB","rd_TN", "rd_fin", "rd_fin1", "rd_fin5"]].sort_values(by="rd_fin5", ascending=False)
